# Music Recommendation - Clustering Oriented

In [7]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sqlite3
import sklearn

# import tools functions
import h5df_getters

Set your environnement configuration here

In [4]:
project_abspath = "/home/osboxes/Documents/python/machlrn/music-recommendation"

## Data

Here is some information about the data.

-> Features for 10000 songs (using the subset of MillionSongDataset)
Download here : http://static.echonest.com/millionsongsubset_full.tar.gz
In order to succesfully access to the data in this huge dataset it is recommended to look at http://millionsongdataset.com/pages/tutorial/. One can try to open file using the h5py library. Then you can take a look at this tutorials to use it :http://docs.h5py.org/en/stable/quick.html#quick.

-> Listener tastes dataset. A triplet-shaped dataset (user_id, song_id, play_count)
Download here : http://millionsongdataset.com/sites/default/files/challenge/train_triplets.txt.zip

-> Access to the songs styles (MSD Allmusic Genre Dataset (Top MAGD)). We used the smallest one with 13 different styles.
The access online is direct without download : http://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-topMAGD-genreAssignment.cls

In [9]:
link_songs_styles = "http://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-topMAGD-genreAssignment.cls"
SONGS_STYLES = pd.read_csv(link_songs_styles, sep="\t")

In [14]:
path_to_user_tastes_dataset = os.path.join(project_abspath, "data/train_triplets.txt")
SONGS_TASTES = pd.read_csv(path_to_user_tastes_dataset, sep="\t")

In [69]:
path_to_info_files = os.path.join(project_abspath, "data/MillionSongSubset/AdditionalFiles")

# connect to database
database_track_metadata = "subset_track_metadata.db"
db_track_metadata = sqlite3.connect(os.path.join(path_to_info_files, database_track_metadata))
# db_track_metadata.execute("SELECT * FROM sqlite_master").fetchall()

In [25]:
path_to_songs_features_data = os.path.join(project_abspath, "data/MillionSongSubset/data")

list_features_name = ['bars_confidence', 'bars_start', 'beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches', 'segments_start', 'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start']

def get_feature(basedir, fun=h5df_getters.get_title, ext='.h5', nb_elements=10000):
    stock = []
    i = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            h5 = h5df_getters.open_h5_file_read(f)
            stock.append(fun(h5))
            i += 1
            h5.close()
            if i > nb_elements-1:
                return stock
    return stock

get_all_titles(basedir=path_to_songs_features_data, nb_elements=5)

[b"Ma' Be Easy [Explicit Version]",
 b'I WANNA HEAR IT FROM YOU',
 b'Oblivious Darkness',
 b'Party For Pink',
 b'Only You (Chimi-Changa Club Mix)']